本 notebook 对比当各个detector选取自己最大的disagreement点做阈值的时候，是否能够起到如下两个作用：
1. 提高f1
2. 甄别 detector

In [3]:
from __future__ import division,print_function
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import re
from sklearn.preprocessing import RobustScaler
import scipy.stats as ss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import scipy.io
import os
import sys
from time import time
import scipy.stats as ss
from sklearn.preprocessing import RobustScaler
from sklearn import metrics

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
# Import all models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.lscp import LSCP # ensemble
import multiprocessing as mp
from IPython.display import display
from sklearn.metrics import roc_auc_score
from thresholders import *


data_path = "/Users/kadima/experiment_any/anomaly-detection/datasets/"
def getData(fileName):
    # check fileName:
    files = [x for x in os.listdir(data_path) if x.endswith(".mat")]
    mat = scipy.io.loadmat(data_path+fileName)
    X = mat["X"]
    y = mat["y"]
    return X, y

def get_score_matrix(X, num_detectors):
    return np.zeros([X.shape[0], num_detectors])

def get_perform_matrix(num_thresholders, num_detectors):
    return np.zeros((num_thresholders, num_detectors))


random_state = np.random.RandomState(10)
outliers_fraction = 0.4
# initialize a set of detectors for LSCP
detector_list = [LOF(n_neighbors=5), LOF(n_neighbors=10), LOF(n_neighbors=15),
                 LOF(n_neighbors=20), LOF(n_neighbors=25), LOF(n_neighbors=30),
                 LOF(n_neighbors=35), LOF(n_neighbors=40), LOF(n_neighbors=45),
                 LOF(n_neighbors=50)]
classifiers = {
    'Angle-based Outlier Detector (ABOD)':
        ABOD(contamination=outliers_fraction),
#     'Cluster-based Local Outlier Factor (CBLOF)':
#         CBLOF(contamination=outliers_fraction,
#               check_estimator=False, random_state=random_state),
    'Feature Bagging':
        FeatureBagging(LOF(n_neighbors=35),
                       contamination=outliers_fraction,
                       random_state=random_state),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=random_state, n_estimators = 280),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers_fraction),
    'Average KNN': KNN(method='mean',
                       contamination=outliers_fraction),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers_fraction),
#     'Minimum Covariance Determinant (MCD)': MCD(
#         contamination=outliers_fraction, random_state=random_state),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
#     'Principal Component Analysis (PCA)': PCA(
#         contamination=outliers_fraction, random_state=random_state),
#     'Locally Selective Combination (LSCP)': LSCP(
#         detector_list, contamination=outliers_fraction,
#         random_state=random_state)
}

names = []
# Show all detectors
for i, clf in enumerate(classifiers.keys()):
    names.append(clf)
#     print('Model', i + 1, clf)

# Fit the models with the generated data and 
# compare model performances
def get_result(X, y, classifiers):
    threshold_records = list()
    # create matrix to store the performance
    score_matrix = get_score_matrix(X, len(classifiers.keys()))
    perform_table = get_perform_matrix(6, len(classifiers.keys()) )

    np.random.seed(5)
    clfs = []

    # Fit the model
    for i, (clf_name, clf) in enumerate(classifiers.items()):
#         print(i + 1, 'fitting', clf_name)
        # fit the data and tag outliers
        clf.fit(X)
        clfs.append(clf)
        scores_pred = clf.decision_function(X)
        score_matrix[:, i] = scores_pred
        

    for i, thresholder in enumerate([sd_thresholder, mad_thresholder, 
                                     iqr_thresholder]):
        kk = []
        for j in range(score_matrix.shape[1]):
            _,perform_table[i,j],b = thresholder(score_matrix[:,j], y)
            kk.append(b)
            
        threshold_records.append(kk)
            
    for i in range(score_matrix.shape[1]):
        perform_table[-3, i] = f1_score(y, clfs[i].predict(X))
        perform_table[-2, i] = roc_auc_score(y, clf.predict_proba(X)[:, 1])

        
    perform_table[-1,:], d_threshold, disagreement_record = disagreement_one_scalar(score_matrix, len(classifiers), y, False)
    threshold_records.append(d_threshold)
    
    return (pd.DataFrame(perform_table, columns = names, 
            index = ["sd",'mad','iqr','default-f1','auc','disagreement']), 
            threshold_records, disagreement_record, score_matrix)

def run_helper(datasets):
    X,y = getData(datasets)
    X = X.astype(np.float64)
    return get_result(X, y, classifiers)


result_dict = dict()
bug_datasets = ["letter.mat","speech.mat","cardio.mat","http.mat", "smtp.mat",
               "ecoli.mat","shuttle.mat","seismic-bumps.arff","mammography.mat","cover.mat"]
for datasets in os.listdir("/Users/kadima/experiment_any/anomaly-detection/datasets/"):
    if datasets in bug_datasets:
        continue
    print(datasets)
    result_dict[datasets] = run_helper(datasets)


display(result_dict["lympho.mat"][0])


pima.mat
wine.mat
mnist.mat


/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


arrhythmia.mat


/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


pendigits.mat
musk.mat
thyroid.mat


...

satellite.mat
lympho.mat
annthyroid.mat


/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/lib/python3.8/site-packages

ionosphere.mat
wbc.mat
glass.mat
satimage-2.mat
vertebral.mat


/opt/anaconda3/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (10.756303907296980 > -6.541214171251153). You may want to try with a higher value of support_fraction (current value: 0.517).
  warnings.warn("Determinant has increased; this should not happen: "


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,0.000000,0.666667,0.800000,0.909091,0.600000,0.600000,0.666667,0.500000,0.400000
mad,0.000000,0.077922,0.000000,0.909091,0.077922,0.077922,0.077922,0.218182,0.545455
iqr,0.000000,0.714286,0.857143,0.769231,0.500000,0.533333,0.666667,0.500000,0.545455
default-f1,0.166667,0.187500,0.184615,0.184615,0.342857,0.500000,0.184615,0.184615,0.184615
auc,0.977700,0.977700,0.977700,0.977700,0.977700,0.977700,0.977700,0.977700,0.977700
disagreement,0.296296,0.285714,0.800000,0.909091,0.476190,0.555556,0.285714,0.235294,0.444444


In [6]:
for datasets in os.listdir("/Users/kadima/experiment_any/anomaly-detection/datasets/"):
    if datasets in bug_datasets:
        continue
    else:
        print(datasets)
        print(np.sum(result_dict[datasets][2],axis=0))
        idx_worst = np.argmax(np.median(result_dict[datasets][2],axis=0))
        idx_best = np.argmin(np.median(result_dict[datasets][2],axis=0))

#         print(names[idx_worst],'is worst')
#         print(names[idx_best], 'is best')
        display(result_dict[datasets][0])
        display(pd.DataFrame(np.asarray(result_dict[datasets][1]).reshape(-1,9),
                             columns=result_dict[datasets][0].columns, 
                             index = ['sd', 'mad', 'iqr','disagreement']))
        print("=="*55)
        print("\n\n")


pima.mat
525.4917015719627


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,0.000000,0.063380,0.095563,0.070671,0.036101,0.043321,0.056140,0.115254,0.000000
mad,0.000000,0.517375,0.517375,0.057143,0.465649,0.472441,0.517375,0.364066,0.000000
iqr,0.000000,0.158824,0.126667,0.057143,0.155340,0.118421,0.171254,0.234957,0.000000
default-f1,0.481802,0.422383,0.528696,0.566957,0.475836,0.379391,0.441652,0.553043,0.420870
auc,0.539302,0.539302,0.539302,0.539302,0.539302,0.539302,0.539302,0.539302,0.539302
disagreement,0.014815,0.014815,0.532020,0.022059,0.007435,0.007435,0.007435,0.007435,0.500566


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,3.898839e-05,1.609722,23.108828,0.118350,57.780390,38.123400,1.612252,59.229399,0.000733
mad,8.458347e-06,0.212685,2.817009,0.125791,21.504689,15.494152,0.240251,19.780183,0.001121
iqr,1.200092e-05,1.276634,22.778228,0.124183,38.740957,28.037795,1.293413,32.865689,0.001047
disagreement,-1.441992e-10,2.914823,20.243166,0.159419,270.980410,161.994200,3.231154,153.184688,-0.000239





wine.mat
107.84306065141558


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,0.000000,0.666667,0.000000,0.000000,0.666667,0.750000,0.666667,0.588235,0.000000
mad,0.000000,0.143885,0.143885,0.000000,0.425532,0.400000,0.143885,0.400000,0.095238
iqr,0.000000,0.645161,0.000000,0.000000,0.833333,0.769231,0.645161,0.636364,0.105263
default-f1,0.333333,0.333333,0.258065,0.258065,0.392157,0.526316,0.333333,0.322581,0.096774
auc,0.475210,0.475210,0.475210,0.475210,0.475210,0.475210,0.475210,0.475210,0.475210
disagreement,0.500000,0.259740,0.000000,0.000000,0.500000,0.588235,0.157480,0.166667,0.114943


...

,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,0.000000,0.074627,0.476190,0.589286,0.292035,0.270742,0.079710,0.611354,0.000000
mad,0.000000,0.024174,0.024174,0.409496,0.031640,0.027434,0.024174,0.077216,0.000000
iqr,0.000000,0.060150,0.299020,0.350254,0.203297,0.185792,0.061758,0.194787,0.000000
default-f1,0.041651,0.020539,0.058528,0.059365,0.063120,0.111226,0.021432,0.059365,0.020067
auc,0.420907,0.420907,0.420907,0.420907,0.420907,0.420907,0.420907,0.420907,0.420907
disagreement,0.090476,0.039070,0.027778,0.027778,0.233766,0.189055,0.037234,0.404040,0.022468


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,2.482073e-07,1.481917,114.278031,0.110183,46.146462,33.522359,1.485320,483.106733,0.000500
mad,2.813051e-07,0.272503,2.910328,0.085146,18.662439,13.187549,0.268408,54.563300,0.000448
iqr,1.756973e-07,1.389678,113.270786,0.077847,40.879043,29.914119,1.379072,111.582013,0.000523
disagreement,-2.443549e-08,1.219851,119.127767,0.300472,42.544095,29.379102,1.209861,1271.701557,-0.000365





vertebral.mat
150.86201952862595


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mad,0.000000,0.222222,0.222222,0.000000,0.114286,0.079365,0.222222,0.027397,0.000000
iqr,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000
default-f1,0.125984,0.183333,0.111111,0.111111,0.088496,0.025316,0.177419,0.126984,0.269841
auc,0.625794,0.625794,0.625794,0.625794,0.625794,0.625794,0.625794,0.625794,0.625794
disagreement,0.000000,0.000000,0.039216,0.000000,0.000000,0.000000,0.000000,0.000000,0.264706


,Angle-based Outlier Detector (ABOD),Feature Bagging,Histogram-base Outlier Detection (HBOS),Isolation Forest,K Nearest Neighbors (KNN),Average KNN,Local Outlier Factor (LOF),Minimum Covariance Determinant (MCD),One-class SVM (OCSVM)
sd,3.971734e-05,2.152736,19.228757,0.143777,71.473083,53.479981,2.241742,138.602298,0.000629
mad,1.173622e-05,0.235108,0.906859,0.102229,16.775620,12.556720,0.228372,16.113625,0.000608
iqr,1.605367e-05,1.300725,19.236822,0.117639,34.385352,24.838450,1.292260,22.683827,0.000501
disagreement,-1.188478e-13,7.026778,18.948980,0.375282,318.116255,245.874570,7.624551,742.927464,0.000124


表现好的